In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

!pip install torch
!pip install torchvision torchaudio

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# dir = 'C:/Users/justi/Downloads/auto_price.csv'
# auto_price = pd.read_csv(dir)

In [ ]:
# target_hp = auto_price[['target', 'horsepower']]
# target_hp

In [ ]:
import torch

x = torch.randn(1, 3, 1, 4)
print(f"Original shape: {x.shape}") 

y = torch.squeeze(x)
print(f"Shape after squeezing all dimensions of size 1: {y.shape}")

z = torch.squeeze(x, dim=0) # Squeeze the first dimension (index 0)
print(f"Shape after squeezing dim 0: {z.shape}") 

print(f"This is what x looks like: \n {x}")
print(f"This is what y looks like: \n {y}")
print(f"This is what z looks like: \n {z}")

### PyTorch Optimization loop song!
It's train time!

do the forward pass,

calculate the loss

optimizer zero grad

losssssss backwards!

Optimizer step step step

Let's test now!

with torch no grad:

do the forward pass,

calculate the loss,

watch it go down down down!

**This song is not helpful**

In [ ]:
loss_fn = torch.nn.L1Loss()
epochs = 1
for epoch in range(epochs):
    # forward pass
    y_pred = model(X_train)

    # calculate loss
    loss = loss_fn(y_pred, y_train)

    # optimizer zero grad
    optimizer.zero_grad()

    # loss backwards
    loss.backward()

    # optimizer step
    optimizer.step()

In [ ]:
do the forward step
calculate the loss
optimizer zero grad
losssssss backward
optimizer step step step

### Make circles and make nn to predict

In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

### -------------------------------------------------------------------------------
class model(nn.Module):
    def __init__(self, n_inputs, n_outputs, hidden_units):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=n_inputs, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=n_outputs)
        )
    
    def forward(self, x):
        return self.linear_layer_stack(x)
### -------------------------------------------------------------------------------

X, y = make_circles(n_samples=1000,
                    random_state=77)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)
X_train, X_test = torch.from_numpy(X_train).type(torch.float32), torch.from_numpy(X_test).type(torch.float32)
y_train, y_test = torch.from_numpy(y_train).type(torch.float32), torch.from_numpy(y_test).type(torch.float32)
# X_train.dtype, X_test.dtype, y_train.dtype, y_test.dtype
# (torch.float64, torch.float64, torch.int64, torch.int64)

# Make optimizer and loss function and model
my_model = model(n_inputs=2, n_outputs=1, hidden_units=8)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=my_model.parameters(), lr=0.1)

epochs = 100
loss_list = []
for epoch in range(epochs):
    my_model.train()

    y_logits = my_model(X_train).squeeze()
    # y_pred_prob = torch.sigmoid(y_logits)
    # y_preds = torch.round(y_pred_prob)

    loss = loss_fn(y_logits, y_train)
    
    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    # Testing
    my_model.eval()
    with torch.inference_mode():
        test_logits = my_model(X_test).squeeze()
        test_loss = loss_fn(test_logits, y_test)
        loss_list.append(test_loss)

    # if epoch % 100 == 0 :
    #     print(f"Epoch {epoch} | Train Loss: {loss} | Test Loss: {test_loss}")



loss_list = [t.numpy() for t in loss_list]
loss_list = np.stack(loss_list)

plt.figure(figsize=(6, 6))
plt.subplot(2, 1, 1)
sns.lineplot(x=np.arange(start=0, stop=len(loss_list)), y=loss_list)
plt.subplot(2, 1, 2)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y)

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!nvidia-smi

In [ ]:
%pip install pytorch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 pytorch-cuda=11.7 -c pytorch -c nvidia


In [ ]:
%pip install --upgrade torch torchvision torchaudio \
    --index-url https://download.pytorch.org/whl/cu118
### THIS CODE HELPED ME INSTALL GPU
# My problem aparently was that I downloaded the pytorch without cuda so it never actually worked

Looking in indexes: https://download.pytorch.org/whl/cu118
   ---------------------------------------- 0.0/2.8 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 GB 4.8 MB/s eta 0:09:46
   ---------------------------------------- 0.0/2.8 GB 6.9 MB/s eta 0:06:51
   ---------------------------------------- 0.0/2.8 GB 7.0 MB/s eta 0:06:43
   ---------------------------------------- 0.0/2.8 GB 7.0 MB/s eta 0:06:43
   ---------------------------------------- 0.0/2.8 GB 7.3 MB/s eta 0:06:25
   ---------------------------------------- 0.0/2.8 GB 7.3 MB/s eta 0:06:26
   ---------------------------------------- 0.0/2.8 GB 7.3 MB/s eta 0:06:22
   ---------------------------------------- 0.0/2.8 GB 7.3 MB/s eta 0:06:27
   ---------------------------------------- 0.0/2.8 GB 7.6 MB/s eta 0:06:09
   ---------------------------------------- 0.0/2.8 GB 7.4 MB/s eta 0:06:18
   ---------------------------------------- 0.0/2.8 GB 7.6 MB/s eta 0:06:10
   ---------------------------------